<a href="https://colab.research.google.com/github/Lauralethia/RedesAge/blob/master/Age_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CNN para predecir edad a partir de volúmenes MRI-T1
Noviembre 2019
Laura Alethia de la Fuente & Mauro Namías

In [0]:
import torch
from torch.autograd import Variable
from torch import autograd
from torch.utils import data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader
from scipy import ndimage as nd
import sys

#import imageio # To save images as gif
import torch.utils.data as d_u
from sklearn.model_selection import train_test_split

Definición de la CNN

In [0]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3x3 square convolution
        # kernel
        self.conv1 = nn.Conv3d(1, 8, 3, padding=1)
        self.conv2 = nn.Conv3d(8, 8, 3, stride = 2, padding=0)
        
        self.conv3 = nn.Conv3d(8, 16, 3, padding=1)
        self.conv4 = nn.Conv3d(16, 16, 3, stride = 2, padding=0)
        
        self.conv5 = nn.Conv3d(16, 32, 3, padding=1)
        self.conv6 = nn.Conv3d(32, 32, 3, padding=1)
        self.conv7 = nn.Conv3d(32, 32, 3, stride =2 ,padding=0)
        
        self.conv8 = nn.Conv3d(32, 64, 3, padding=1)
        self.conv9 = nn.Conv3d(64, 64, 3, padding=1)
        self.conv10 = nn.Conv3d(64, 64, 3, stride = 2, padding=0)
      
        self.fc1 = nn.Linear(128, 128)
        self.bn1 = nn.BatchNorm1d(128,momentum=0.1)
        self.do = nn.Dropout(p = 0.7, inplace=False)        
        self.fc2 = nn.Linear(128, 64)        
        self.fc3 = nn.Linear(64,1)
        
     
   
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.elu(self.conv1(x))
        x = F.elu(self.conv2(x))        
        x = F.elu(self.conv3(x))
        x = F.elu(self.conv4(x))
        x = F.elu(self.conv5(x))
        x = F.elu(self.conv6(x))
        x = F.elu(self.conv7(x))
        x = F.elu(self.conv8(x))
        x = F.elu(self.conv9(x))
        x = F.elu(self.conv10(x))
        
        x = x.view(-1, self.num_flat_features(x))
#        print(x)
        x = self.fc1(x)
        x = F.elu(x)
        x = self.bn1(x)
        x = self.do(x)        
        x = self.fc2(x)
        x = F.elu(x)
        x = self.fc3(x) 

        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

Probamos que haya quedado bien definida la red

In [0]:
## Inic.
net = Net()
print(net)

Net(
  (conv1): Conv3d(1, 8, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv2): Conv3d(8, 8, kernel_size=(3, 3, 3), stride=(2, 2, 2))
  (conv3): Conv3d(8, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv4): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(2, 2, 2))
  (conv5): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv6): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv7): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2))
  (conv8): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv9): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv10): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2))
  (fc1): Linear(in_features=128, out_features=128, bias=True)
  (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (do): Dropout(p=0.7, inplace=False)
  (fc2): Linear(in_featur

Probamos la red en CPU

In [0]:
input = Variable(torch.randn(1,1, 46,52,40))
#print(input)
net.eval()
output = net(input)
print(output)


tensor([[0.0274]], grad_fn=<AddmmBackward>)


Probamos la red en GPU

In [0]:
input = Variable(torch.randn(1,1, 46,52,40))
net.eval()

use_cuda = torch.cuda.is_available()
print(use_cuda)

if use_cuda:
    net = net.cuda()
    input = input.cuda()

tic = time.clock()
output = net(input)
print(output)
toc = time.clock()
print(toc-tic)

False
tensor([[0.0272]], grad_fn=<AddmmBackward>)
0.07740599999999986


In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Load functions and check datasets

#!ls "/content/gdrive/My Drive/Doctorado/Age_CNN/data"
!ls "/content/gdrive/My Drive/COCUCO/Redes/Datos"

AB_CALTECH_0051478_bet1_V.nii  AB_CMU_0050656_bet1_V.nii
AB_CALTECH_0051479_bet1_V.nii  AB_CMU_0050657_bet1_V.nii
AB_CALTECH_0051480_bet1_V.nii  AB_CMU_0050658_bet1_V.nii
AB_CALTECH_0051481_bet1_V.nii  AB_CMU_0050659_bet1_V.nii
AB_CALTECH_0051482_bet1_V.nii  AB_CMU_0050660_bet1_V.nii
AB_CALTECH_0051483_bet1_V.nii  AB_CMU_0050661_bet1_V.nii
AB_CALTECH_0051484_bet1_V.nii  AB_CMU_0050663_bet1_V.nii
AB_CALTECH_0051485_bet1_V.nii  AB_CMU_0050664_bet1_V.nii
AB_CALTECH_0051486_bet1_V.nii  AB_CMU_0050665_bet1_V.nii
AB_CALTECH_0051487_bet1_V.nii  AB_CMU_0050666_bet1_V.nii
AB_CALTECH_0051488_bet1_V.nii  AB_CMU_0050668_bet1_V.nii
AB_CALTECH_0051489_bet1_V.nii  alldata_2c.csv
AB_CALTECH_0051492_bet1_V.nii  reducidoColab.csv
AB_CALTECH_0051493_bet1_V.nii


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# Net parameters and files
#!ls "/content/drive/My Drive/COCUCO/Redes/Datos"
csvfDir = '/content/drive/My Drive/COCUCO/Redes/Datos'
csvfName = 'reducido.csv'

folds = 2
nEpochs = 400
bs = 20 # Bach size for training 

AB_CALTECH_0051478_bet1_V.nii  AB_CALTECH_0051493_bet1_V.nii
AB_CALTECH_0051479_bet1_V.nii  AB_CMU_0050656_bet1_V.nii
AB_CALTECH_0051480_bet1_V.nii  AB_CMU_0050657_bet1_V.nii
AB_CALTECH_0051481_bet1_V.nii  AB_CMU_0050658_bet1_V.nii
AB_CALTECH_0051482_bet1_V.nii  AB_CMU_0050659_bet1_V.nii
AB_CALTECH_0051483_bet1_V.nii  AB_CMU_0050660_bet1_V.nii
AB_CALTECH_0051484_bet1_V.nii  AB_CMU_0050661_bet1_V.nii
AB_CALTECH_0051485_bet1_V.nii  AB_CMU_0050663_bet1_V.nii
AB_CALTECH_0051486_bet1_V.nii  AB_CMU_0050664_bet1_V.nii
AB_CALTECH_0051487_bet1_V.nii  AB_CMU_0050665_bet1_V.nii
AB_CALTECH_0051488_bet1_V.nii  AB_CMU_0050666_bet1_V.nii
AB_CALTECH_0051489_bet1_V.nii  AB_CMU_0050668_bet1_V.nii
AB_CALTECH_0051492_bet1_V.nii  alldata_2c.csv


In [0]:
# Training and test run
kfold_loss = []
kfold_loss_t = []

for k in range(folds):
    net = Net()
    net = net.cuda()
    alldata = MRIDataset(csv_file=str(csvfDir+csvfName), root_dir=csvfDir)
    tot = len(alldata)
    alldataloader = DataLoader(alldata, batch_size=tot, shuffle=True, num_workers=0) 
    for i, data in enumerate(alldataloader):
            inputs, labels = data
            inputs = Variable(data[inputs].float(),requires_grad=True)
            inputs = inputs.squeeze(1)
            labels = Variable(data[labels].float())
            labels = labels.squeeze(1)
    
    dataset_x = inputs.data.numpy()
    dataset_y = labels.data.numpy().astype('float64')     
    
    X_train, X_test, y_train, y_test = train_test_split(dataset_x,dataset_y,test_size=0.33,
                                                        random_state=42)    

    my_sampler =d_u.sampler.SubsetRandomSampler(list(range(len(X_train))))
    my_sampler2 =d_u.sampler.SubsetRandomSampler(list(range(len(X_test))))

    X_train[X_train<0] = 0
    X_test[X_test<0] = 0   
    
    test_largo = len(y_test)
    train_largo = len(y_train)
      
  #samples_weight = np.ones([1,train_largo])
  #weight = weight/weight.sum()
  #samples_weight = np.array([weight[t] for t in y_train])
  #samples_weight = samples_weight/3.0
  #samples_weight = torch.from_numpy(samples_weight)  
  # sampler = torch.utils.data.sampler.WeightedRandomSampler(samples_weight.type('torch.DoubleTensor'), len(samples_weight),replacement=False)
    trainDataset = d_u.TensorDataset(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
    validDataset = d_u.TensorDataset(torch.FloatTensor(X_test), torch.FloatTensor(y_test)) 
    del X_train, X_test
    
    bs_test = round(len(y_test)/2)
    
    trainLoader = d_u.DataLoader(dataset = trainDataset, batch_size=bs, num_workers=0, sampler = my_sampler)
#    testLoader = d_u.DataLoader(dataset = validDataset, batch_size=bs_test, shuffle=True, num_workers=0) 
    testLoader = d_u.DataLoader(dataset = validDataset, batch_size=bs_test, num_workers=0,sampler = my_sampler2) 
   
    net.train()
       
    lr = 0.01  # 0.1
    wd = 0.01  # 0.01

    optimizer = optim.SGD(filter(lambda p: p.requires_grad, net.parameters()), lr= lr ,momentum=0.1, weight_decay=wd)
    criterion = nn.MSELoss()

# One result figure by kfols
    fig = plt.figure()
    
    batch_loss = []
    batch_loss_t = []
       
    for epoch in range(nEpochs):  # loop over the dataset multiple times
        print("learning rate")
        print(lr)
        print("weight decay")
        print(wd)
        optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.1, weight_decay=wd)    
        net.train()
        mini_batch_loss = []
        mini_batch_loss_t = []
       
        for i, data in enumerate(trainLoader):
            inputs2,labels2 = data
            dataumentada = dataugment(Variable(inputs2),Variable(labels2) )         
            instanceloader = DataLoader(dataumentada, batch_size=len(labels), shuffle=False, num_workers=0)
    
            for i_a, data_a in enumerate(instanceloader):
                inputs2, labels2 = data_a
                inputs2 = data_a[inputs2]
                labels2 = data_a[labels2]
                
            tici = time.clock()
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward + backward + optimize
            inputs2 = Variable(inputs2)
            labels2 = Variable(labels2).float()

            outputs = net(inputs2.cuda())            

            loss = criterion(outputs.cpu(),labels2)                   
            loss = loss.cuda()
            loss.backward()
            optimizer.step()
            mini_batch_loss.append(loss.data[0])
        batch_loss.append(np.array(mini_batch_loss).mean())

        toco = time.clock()
    
    #    torch.save(net.state_dict(), 'C:\\Users\\Amelie\\Documents\\Cursos\\Redes\\Proyecto\\Mau\\REDES\\T1ImgBet\\entrenadafullrot3.dat')
    #    torch.save(net.state_dict(), 'C:\\Users\\Amelie\\Documents\\Cursos\\Redes\\Proyecto\\Mau\\REDES\\T1ImgVBM\\entrenada.dat')       
        toc = time.clock()  
    
        print('Finished Epoch')
        print(epoch)
        
        net.eval()
        average_t_loss = 0
                    
        for i, data_t in enumerate(testLoader):
                # get the inputs
                tici = time.clock()
                t_inputs2, t_labels2 = data_t
                if t_labels2.size()[0] > 1:
                    t_inputs2 = Variable(t_inputs2.float(),volatile = True)
                    t_labels2 = Variable(t_labels2).float()
            
                    t_outputs = net(t_inputs2.cuda())
                    t_loss = criterion(t_outputs.cpu(),t_labels2)
                    mini_batch_loss_t.append(t_loss.data[0])
        batch_loss_t.append(np.array(mini_batch_loss_t).mean())
          
# Result figures by fold          
        plt.ion()
        ax1 = fig.add_subplot(211)    # The big subplot
        ax2 = fig.add_subplot(212)
        
        ax1.plot(batch_loss,color='y')
        
        x = np.array(batch_loss_t)
        idx = [np.where(x >1000)]
        x[idx] -= x[idx]
        ax1.plot(x,color='g')
        ax1.set_ylabel('Loss promedio train(y)/test(g)')
        ax1.set_xlabel('Epoch')

        ax2.scatter(labels2.data.numpy(),outputs.cpu().data.numpy() ,color='red', alpha=1*(epoch/nEpochs))
        ax2.scatter(t_labels2.data.numpy(),t_outputs.cpu().data.numpy() ,color='orange', alpha=1*(epoch/nEpochs))
        ax2.set_ylabel('Dep/PredAge')
        ax2.set_xlabel('Indep/Age [y = test ; r = train]')
        ax2.set_ylim([15, 35])
        ax2.set_xlim([15 ,35])
        
        fig.show()
        plt.pause(0.05)
  #          redfoldnale = 'C:\\Users\\Amelie\\Documents\\Alethianet\\Entrenada\\entrenada_aletheia_2019_drop08' +str(k) +'.dat'
   #         torch.save(net.state_dict(), redfoldnale)
#        print('average test MSE: ' + str(round((batch_loss_t),5)))
        
        toc = time.clock()
        toc-tic
        print('Finished Training')
        print(toc-tic)
    
 #   foldname = 'C:\\Users\\Amelie\\Documents\\Alethianet\\Entrenada\\' + 'trainfold_2019_drop08' + str(k)+ '.png'
  #  plt.savefig(foldname)
   # plt.close()
kfold_loss.append(np.array(batch_loss).mean())
kfold_loss_t.append(np.array(batch_loss_t).mean())